# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [17]:
#number of applications for each race
wh_total = len(data[data.race=='w'])
bl_total = len(data[data.race=='b'])

print('Total number of applicants is {} for white-sounding and {} for black-sounding.'.format(wh_total, bl_total))

# number of callbacks for white- and black-sounding names
wh = sum(data[data.race=='w'].call)
bl = sum(data[data.race=='b'].call)

print('Number of callbacks for white-sounding is {} and number for black-sounding is {}.'.format(wh, bl))

Total number of applicants is 2435 for white-sounding and 2435 for black-sounding.
Number of callbacks for white-sounding is 235.0 and number for black-sounding is 157.0.


In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [18]:
#call_race = data[['id', 'race', 'call']]
#call_race.groupby('race').sum()

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1. Since this problem involves comparing two different proportions, the proportion of white-sounding applicants that received callbacks vs. those with black-sounding applicants, a two-sample test is appropriate. The proportions of the two respective groups will be compared and then tested for significance.<br>
<br>
The question is whether or not a call-back was received, yes or no, making this a bernoulli problem. For such a problem, the CLT does apply. If the test were repeated several times, the distributions of the proportions observed would approach normal. 

2. The null hypothesis is that there is no difference between call-back rates for black and white applicants, and that any difference observed is merely by chance. The alternate hypothesis, therefore, is that the difference observed is not by chance and black applicants receive lower call-back rates. This would indicate that discrimination is prevalent.

In [20]:
w = data[data.race=='w']
b = data[data.race=='b']

#### Question 3: Computing margin of error, CI, p-value

Frequentist Approach:

In [33]:
#Find the mean for each group and compute the difference
wh_mean = wh / len(w)
bl_mean = bl / len(b)
obs_diff_of_mean = wh_mean - bl_mean

#Compute the std deviation of the distribution of the difference of means
wh_var = wh_mean*(1-wh_mean) / 2435
bl_var = bl_mean*(1-bl_mean) / 2435
diff_std = np.sqrt(wh_var + bl_var)

print('For the distribution of differences, the mean is {} and the std dev is {}.'.\
      format(obs_diff_of_mean, diff_std))

For the distribution of differences, the mean is 0.032032854209445585 and the std dev is 0.0077833705866767544.


Now with the standard deviation calculated, a confidence interval and margin of error can be computed. For this, a 95% confidence interval will be used.

In [52]:
ci = stats.norm.interval(0.95, loc=obs_diff_of_mean, scale=diff_std)

#capture upper and lower bounds
low, up = ci

print('With 95% confidence, the true difference in proportion of call back rates is between {} and {}.'\
      .format(low, up))

#Calculate the margin of error
margin = up - obs_diff_of_mean

print('The margin of error is plus/minus {}.'.format(margin))

With 95% confidence, the true difference in proportion of call back rates is between 0.016777728181230755 and 0.04728798023766041.
The margin of error is plus/minus 0.015255126028214827.


Now to calculate the p-value for the observed result:<br>
To do this, it is assumed there is no true difference between the two groups. The standard deviation under this assumption needs to be calculated in order to compute the z-stat. That means, if there is no difference, the proportion of call-backs is equivalent to the number of call-backs for the entire sample, regardless of race. So *n of call-backs / total applicants* is the mean under the assumption.

In [41]:
#Calculate the assumed proportion and std deviation
assumed = (wh + bl) / len(data)
assumed_std = np.sqrt((assumed*(1-assumed) / len(data)) + (assumed*(1-assumed) / len(data)))

#With the std deviation, now calculate the z-score
assumed_z = obs_diff_of_mean / assumed_std

#Calculate the one-sided p-value
p_val = stats.norm.sf(abs(assumed_z))

print(p_val)

3.120431080512834e-09


Now test with hacker statistics:

In [51]:
#Recalculate 10,000 new mean differences randomly 
#and see how often we get a result as extreme as the one observed
perm_diffs = np.empty(10000)

for i in range(10000):
    comb = np.concatenate((w.call, b.call))
    permed = np.random.permutation(comb)
    perm_white = permed[:len(w)]
    perm_black = permed[len(w):]
    perm_diffs[i] = (np.sum(perm_white) / len(perm_white)) - (np.sum(perm_black) / len(perm_black))

#The observed difference indicates male temps are lower
#therefore, the p-value tests for values as low or lower
p = np.sum(perm_diffs >= obs_diff_of_mean) / len(perm_diffs)

print('P-value is ', p)

P-value is  0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

4. The calculated p-value from above is extremely low, strongly indicating that the null hypothesis should be rejected. From this result, it seems likely that racial bias does factor into the hiring process.

5. As for whether or not race/name is the most important factor in call-back success, I wouldn't necessarily go that far. The tests above merely address one independent variable in a table of 65 columns. Other factors like education, years of experience and other relevant qualities of the applicants were not addressed. Perhaps a more robust analysis would look at the likelihood of a white applicant receiving a call-back in contrast to a black applicant with better qualifications.